In [1]:
import pandas as pd
import numpy as np

# Cargar el dataset crudo
df_raw = pd.read_csv('../data/infobras.csv', low_memory=False)

print("Dataset crudo cargado.")
df_raw.info()

Dataset crudo cargado.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178257 entries, 0 to 178256
Columns: 113 entries, codigo Entidad to Monto Total devengado del proyecto
dtypes: float64(12), int64(16), object(85)
memory usage: 153.7+ MB


In [2]:
# --- VERSIÓN FINAL DE LA PIPELINE DE LIMPIEZA ---
def clean_infobras_data(df_raw):
    df = df_raw.copy()
    print("Iniciando proceso de limpieza...")
    
    # 1. Estandarizar nombres de columnas
    df.columns = df.columns.str.lower().str.replace(' ', '_', regex=False).str.replace('¿', '', regex=False).str.replace('?', '', regex=False).str.replace('(', '', regex=False).str.replace(')', '', regex=False).str.replace(':', '', regex=False).str.replace('.', '', regex=False).str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

    # 2. Renombrar columnas duplicadas
    rename_dict = {
        'ruc1': 'ruc_supervisor', 'nombre_o_razon_social_de_la_empresa_o_consorcio1': 'razon_social_supervisor',
        'monto_del_contrato__en_soles1': 'monto_contrato_soles_supervisor', 'tipo_de_documento_de_identidad1': 'tipo_doc_identidad_residente',
        'numero_de_documento1': 'num_doc_residente', 'nombres_apellidos1': 'nombres_apellidos_residente',
        'colegiatura1': 'colegiatura_residente', 'numero_de_colegiatura1': 'num_colegiatura_residente',
        'fecha_inicio_de_labores': 'fecha_inicio_labores_residente', 'fecha_fin__de_labores': 'fecha_fin_labores_residente'
    }
    df = df.rename(columns=rename_dict)

    # 3. Eliminar columnas inútiles
    cols_to_drop = [
        'fecha_de_aprobacion', 'tipo_de_certificado_de_inversion_publica', 'monto_cipril_/cipgn', 
        'fecha_del_cipril_/_cipgn', 'numero_del_cipril_/cipgn', 'otra_marca'
    ]
    df = df.drop(columns=cols_to_drop, errors='ignore')

    # 4. Convertir tipos de datos
    date_cols = [
        'fecha_de_actualizacion', 'fecha_de_aprobacion_del_expediente', 'fecha_inicio_supervision', 'fecha_fin_supervision',
        'fecha_inicio_labores_residente', 'fecha_fin_labores_residente', 'fecha_de_inicio_de_obra', 
        'fecha_finalizacion_programada_de_obra', 'fecha_de_entrega_del_terreno', 'fecha_de_registro_de_avance', 
        'fecha_de_paralizacion', 'fecha_finalizacion_reprogramada_de_obra', 'fecha_de_finalizacion_real', 
        'fecha_de_recepcion', 'fecha_de_aprobacion_de_liquidacion_de_obra', 'fecha_de_transferencia'
    ]
    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], dayfirst=True, errors='coerce')
    
    numeric_object_cols = [
        'monto_viable/aprobado', 'monto_de_aprobacion_de_expediente_tecnico', 'tasa_de_cambio', 'monto_aprobado_en_soles',
        'monto_del_contrato__en_soles', 'monto_contrato_soles_supervisor', 'porcentaje_de_terreno_entregado',
        'avance_fisico_programado_acumulado_%', 'avance_fisico_real_acumulado_%', 'monto_de_valorizacion_programado_acumulado',
        'monto_de_valorizacion_ejecutado_acumulado', 'porcentaje_de_ejecucion_financiera', 'monto_de_ejecucion_financiera_de_la_obra',
        'monto_de_adicionales_de_obra_en_soles', 'monto_de_adicionales_de_supervision_en_soles',
        'monto_de_deductivos_de_obra_en_soles', 'costo_de_la_obra_en_soles', 'monto_total_devengado_del_proyecto'
    ]
    for col in numeric_object_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace('S/.', '', regex=False).str.replace('%', '', regex=False).str.replace(',', '', regex=False).str.strip()
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # 5. Manejar Nulos
    df['causal_de_paralizacion'] = df['causal_de_paralizacion'].fillna('No Paralizada')
    categorical_fillna_unknown = ['departamento', 'provincia', 'distrito', 'modalidad_de_ejecucion_de_la_obra']
    for col in categorical_fillna_unknown:
        if col in df.columns:
            df[col] = df[col].fillna('Desconocido')
    numeric_fillna_zero = ['monto_aprobado_en_soles', 'monto_del_contrato__en_soles', 'costo_de_la_obra_en_soles']
    for col in numeric_fillna_zero:
        if col in df.columns:
            df[col] = df[col].fillna(0)
            
    print("Proceso de limpieza completo!")
    return df

In [3]:
# Ejecutar la pipeline
df_clean = clean_infobras_data(df_raw)

# Exportar el resultado
df_clean.to_csv('../data/infobras_clean_v1.csv', index=False)

print("\nArchivo 'infobras_clean_v1.csv' exportado exitosamente.")
df_clean.info()

Iniciando proceso de limpieza...


C:\Users\GAMERS\AppData\Local\Temp\ipykernel_12020\3124197627.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], dayfirst=True, errors='coerce')


Proceso de limpieza completo!

Archivo 'infobras_clean_v1.csv' exportado exitosamente.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178257 entries, 0 to 178256
Columns: 108 entries, codigo_entidad to monto_total_devengado_del_proyecto
dtypes: datetime64[ns](16), float64(29), int64(16), object(47)
memory usage: 146.9+ MB
